# Example of how to use geosupport and geosupportBatch

moda.py contains two functions (geosupport for single address lookup and geosupportBatch for batch lookup) for interfacing with DCP's desktop GeoSupport. Before using them, you'll need to:

1. Download the Linux version from DCP's [website](https://www1.nyc.gov/site/planning/data-maps/open-data/dwn-gde-home.page)
2. Point the 'path' variable to where the downloaded files live on your computer.
    Note: You'll need to be running Linux to use this. I use Ubuntu running as a VM via VirtualBox hosted on my Windows desktop.

In [1]:
import pandas as pd

import sys
sys.path.insert(0,'/home/deena/projects/ModaCode/')
import moda

## 1. geosupport

single address lookup

In [2]:
moda.geosupport(1,253,'broadway')

[b"*****  Enter 'X' for Extended Work Area:  \n",
 b'\n',
 b'Function 1A GRC = 00\n',
 b'Error Message =                                                                                 \n',
 b'\n',
 b'\n',
 b'[  0]: ---------0---------1---------2---------3---------4---------5---------6---------7\n',
 b'[  1]: 12345678901234567890123456789012345678901234567890123456789012345678901234567890\n',
 b'[  2]:                                                                                 \n',
 b'[  3]: Access Key                       \t 011136101000255000AA \n',
 b'[  4]: Continuous Parity Indicator      \t  \n',
 b'[  5]: Low Housenum Of Key              \t 000251000AA\n',
 b'[  6]: Bbl                              \t 1001347501\n',
 b'[  7]:    Borough Code                  \t 1\n',
 b'[  8]:    Tax Block                     \t 00134\n',
 b'[  9]:    Tax Lot                       \t 7501\n',
 b'[ 10]: Tax Lot Version Number (Nyi)     \t  \n',
 b'[ 11]: Rpad Self-Check Code (Scc) - Bbl \t 7

## 2. geosupportBatch

look up a dataframe worth of addresses

In [2]:
# download PLUTO to use as an example list of addresses
# read in PLUTO from open data, only including a few columns and rows
url = 'https://data.cityofnewyork.us/resource/64uk-42ks.json'
filters = "?$select=bbl,borocode,address,bldgclass,ownername,numbldgs,lotarea\
&$limit=1000"
pluto = pd.read_json(url+filters)

# since pluto doesn't display the address number and street name separately, let's break them out:
pluto['houseNo'] = pluto['address'].str.extract('(^[0-9|-]*)',expand=False)
pluto['street'] = pluto['address'].str.extract('(\s.+$)',expand=False)

pluto.head()

,bbl,borocode,address,bldgclass,ownername,numbldgs,lotarea,houseNo,street
0,2052700001,2,2222 HOLLERS AVENUE,Z9,GRILO ENTERPRISES INC,1.0,5000.0,2222,HOLLERS AVENUE
1,2052860040,2,3512 HEATHCOTE AVENUE,Z9,PEARTREE AUTO WRECKERS INC,1.0,15000.0,3512,HEATHCOTE AVENUE
2,4097640087,4,161 STREET,G7,"HILLSIDE AVENUE DEVELOPMENT, LLC",0.0,8763.0,161,STREET
3,4116950026,4,117-37 LINCOLN STREET,G0,ABRAHAM TONY,0.0,1855.0,117-37,LINCOLN STREET
4,3028010032,3,THOMAS STREET,G7,PEERLESS EQUITIES LLC,0.0,5525.0,,STREET


In [3]:
# call geosuppotBatch with the pluto dataframe. need to define the columns in pluto which contain address info

df = moda.geosupportBatch(pluto, houseNo='houseNo',street='street',boro='borocode')
df.shape
#df[['houseNo','street','BoroCode','BBL','geocodedBBL','geocodedBIN']]

(1000, 11)

In [8]:
# this is the output
df[['address','houseNo','street','borocode','bbl','geocodedBBL','geocodedBIN']].head()

,address,houseNo,street,borocode,bbl,geocodedBBL,geocodedBIN
0,2222 HOLLERS AVENUE,2222,HOLLERS AVENUE,2,2052700001,2052700001,2093880
1,3512 HEATHCOTE AVENUE,3512,HEATHCOTE AVENUE,2,2052860040,2052860040,2129539
2,161 STREET,161,STREET,4,4097640087,Error Message = ' STREET' NOT RECOGNIZED. THER...,Error Message = ' STREET' NOT RECOGNIZED. THER...
3,117-37 LINCOLN STREET,117-37,LINCOLN STREET,4,4116950026,4116950026,4000000
4,THOMAS STREET,,STREET,3,3028010032,Error Message = ' STREET' NOT RECOGNIZED. THER...,Error Message = ' STREET' NOT RECOGNIZED. THER...
